<a href="https://colab.research.google.com/github/MatheusHenriqueDaSilva/data_science/blob/main/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Aplicação do algoritmo de **Árvore de Decisão** para predizer se a pessoa irá sobreviver ao acidente ocorrido no TITANIC

Etapas:
1.   Data prep
2.   Normalização dos dos valores contínuos
3.   one-hot encoding e Label Encoding nas variáveis categóricas
4.   treinamento e otimização de hiperparâmetros


Lembrando que no caso desse algoritmo, não é necess




In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt


In [3]:
test = pd.read_csv('/content/test.csv')
test.shape


(418, 11)

In [4]:
train = pd.read_csv('/content/train.csv')
train.describe()


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [5]:
lista_spec = ['PassengerId', 'Survived','Ticket','Name']
target = train['Survived']
train.drop(lista_spec,axis=1,inplace=True)


In [6]:
def generate_metadata(dataframe):

    metadata2 = pd.DataFrame({
        'nome_variavel': dataframe.columns,
        'tipo': dataframe.dtypes,
        'qt_nulos': dataframe.isnull().sum(),
        'percent_nulos': round((dataframe.isnull().sum() / len(dataframe))* 100,2),
        'cardinalidade': dataframe.nunique(),
    })
    metadata2=metadata2.sort_values(by='tipo')
    metadata2 = metadata2.reset_index(drop=True)

    return metadata2

metadados = generate_metadata(train)
metadados

,nome_variavel,tipo,qt_nulos,percent_nulos,cardinalidade
0,Pclass,int64,0,0.00,3
1,SibSp,int64,0,0.00,7
2,Parch,int64,0,0.00,7
3,Age,float64,177,19.87,88
4,Fare,float64,0,0.00,248
5,Sex,object,0,0.00,2
6,Cabin,object,687,77.10,147
7,Embarked,object,2,0.22,3


In [7]:
# Variáveis numéricas
# Filtrando o DataFrame
missing_cutoff = 70
df_numericas_drop_nulos = metadados[(metadados['percent_nulos'] >= missing_cutoff)]
lista_drop_nulos_numericas = list(df_numericas_drop_nulos.nome_variavel.values)
lista_drop_nulos_numericas

['Cabin']

In [8]:
# retirando lista de variáveis com alto percentual de nulos
train2 = train.drop(axis=1,columns=lista_drop_nulos_numericas)
train2.shape

(891, 7)

In [9]:
# substituindo nulos pela média da variável numérica
train2 = train2.fillna(train2.select_dtypes(include=['float64', 'int64','int32']).mean())
train2.shape

(891, 7)

In [10]:
# substituindo nulos por "Desconhecido" para categóricas
categorical_cols = train2.select_dtypes(include=['object']).columns
train2[categorical_cols] = train2[categorical_cols].fillna('Desconhecido')
train2.shape

(891, 7)

In [11]:
metadados = generate_metadata(train2)
metadados

,nome_variavel,tipo,qt_nulos,percent_nulos,cardinalidade
0,Pclass,int64,0,0.0,3
1,SibSp,int64,0,0.0,7
2,Parch,int64,0,0.0,7
3,Age,float64,0,0.0,89
4,Fare,float64,0,0.0,248
5,Sex,object,0,0.0,2
6,Embarked,object,0,0.0,4


In [12]:
from sklearn.preprocessing import MinMaxScaler

# Instanciando o scaler
scaler = MinMaxScaler()

# Selecionando colunas numéricas
numeric_cols = train2.select_dtypes(include=['float64', 'int64','int32']).columns

# Aplicando a normalização
train2[numeric_cols] = scaler.fit_transform(train2[numeric_cols])
train2.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1.0,male,0.271174,0.125,0.0,0.014151,S
1,0.0,female,0.472229,0.125,0.0,0.139136,C
2,1.0,female,0.321438,0.000,0.0,0.015469,S
3,0.0,female,0.434531,0.125,0.0,0.103644,S
4,1.0,male,0.434531,0.000,0.0,0.015713,S


In [13]:
from sklearn.preprocessing import OneHotEncoder
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

card_cutoff = 2
df_categ_onehot = metadados[(metadados['cardinalidade'] <= card_cutoff) & (metadados['tipo'] == 'object')]
lista_onehot = list(df_categ_onehot.nome_variavel.values)
print('Lista de vars para OneHot Encoding: ',lista_onehot)

# Instanciando o encoder
encoder = OneHotEncoder(drop='first', sparse_output=False)

# Aplicando o one-hot encoding
encoded_data = encoder.fit_transform(train2[lista_onehot])

# Criando um DataFrame com as colunas codificadas, mantendo o índice original
encoded_cols = encoder.get_feature_names_out(lista_onehot)
encoded_df = pd.DataFrame(encoded_data, columns=encoded_cols, index=train2.index)

# Concatenando o DataFrame codificado com o DataFrame original
train3 = pd.concat([train2.drop(lista_onehot, axis=1), encoded_df], axis=1)

train3.head()

Lista de vars para OneHot Encoding:  ['Sex']


,Pclass,Age,SibSp,Parch,Fare,Embarked,Sex_male
0,1.0,0.271174,0.125,0.0,0.014151,S,1.0
1,0.0,0.472229,0.125,0.0,0.139136,C,0.0
2,1.0,0.321438,0.000,0.0,0.015469,S,0.0
3,0.0,0.434531,0.125,0.0,0.103644,S,0.0
4,1.0,0.434531,0.000,0.0,0.015713,S,1.0


In [14]:
card_cutoff = 2
df_categ_labelenc = metadados[(metadados['cardinalidade'] > card_cutoff) & (metadados['tipo'] == 'object')]
lista_lenc = list(df_categ_labelenc.nome_variavel.values)
print('Lista de vars para Label Encoding: ',lista_lenc)


Lista de vars para Label Encoding:  ['Embarked']


In [15]:
from sklearn.preprocessing import LabelEncoder

# Aplicando LabelEncoder nas colunas desejadas
for col in lista_lenc:
    encoder = LabelEncoder()
    train3[col] = encoder.fit_transform(train3[col])

train3.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    float64
 1   Age       891 non-null    float64
 2   SibSp     891 non-null    float64
 3   Parch     891 non-null    float64
 4   Fare      891 non-null    float64
 5   Embarked  891 non-null    int64  
 6   Sex_male  891 non-null    float64
dtypes: float64(6), int64(1)
memory usage: 48.9 KB


In [16]:
train_f = pd.concat([train3, target], axis=1)
train_f.head()

,Pclass,Age,SibSp,Parch,Fare,Embarked,Sex_male,Survived
0,1.0,0.271174,0.125,0.0,0.014151,3,1.0,0
1,0.0,0.472229,0.125,0.0,0.139136,0,0.0,1
2,1.0,0.321438,0.000,0.0,0.015469,3,0.0,1
3,0.0,0.434531,0.125,0.0,0.103644,3,0.0,1
4,1.0,0.434531,0.000,0.0,0.015713,3,1.0,0


feature selection


In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier



X = train_f.drop(axis=1,columns='Survived')
y = train_f.Survived

# Treinar modelo com algoritmo Random Forest (Calma, não precisa focar no modelo neste momento)
clf = RandomForestClassifier() # troque aqui qual classificador quer utilizar: GradientBoostingClassifier, DecisionTreeClassifier
clf.fit(X, y)

# Obter importância das variáveis
feature_importances = clf.feature_importances_
features = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances
})

features


,Feature,Importance
0,Pclass,0.094576
1,Age,0.262632
2,SibSp,0.046710
3,Parch,0.037595
4,Fare,0.265417
5,Embarked,0.032465
6,Sex_male,0.260604


In [18]:
# Ordenar variáveis por importância
features = features.sort_values(by='Importance', ascending=False)

# Estabelecer um limite de corte, por exemplo, x% da importância máxima
cutoff = 0.01

#cutoff = cutoff_maximp * feature_importances.max()

# Selecionar variáveis acima do corte
selected_features = features['Feature'][features['Importance'] > cutoff].tolist()
selected_features

['Fare', 'Age', 'Sex_male', 'Pclass', 'SibSp', 'Parch', 'Embarked']

In [19]:
train_pronta = train_f.loc[:,selected_features]
train_pronta.head()

,Fare,Age,Sex_male,Pclass,SibSp,Parch,Embarked
0,0.014151,0.271174,1.0,1.0,0.125,0.0,3
1,0.139136,0.472229,0.0,0.0,0.125,0.0,0
2,0.015469,0.321438,0.0,1.0,0.000,0.0,3
3,0.103644,0.434531,0.0,0.0,0.125,0.0,3
4,0.015713,0.434531,1.0,1.0,0.000,0.0,3


In [20]:
from sklearn.model_selection import train_test_split

# Dividindo os dados em conjunto de treinamento e teste (70% treino, 30% teste)
X_train, X_test, y_train, y_test = train_test_split(train_pronta,
                                                    train_f.Survived,
                                                    test_size=0.3,
                                                    random_state=42)
X_train.shape,X_test.shape

((623, 7), (268, 7))

In [21]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV


# Definindo o modelo
model = DecisionTreeClassifier(random_state=42)

# Definindo os parâmetros para o grid search
param_grid = {
    'criterion':['gini','entropy'],
    'splitter':['best','random'],
    'max_depth': [2,3,4], # Profundidade máxima da árvore
    'min_samples_split': [0.008, 0.01,0.02,0.04], # Número mínimo de amostras necessárias para dividir um nó interno.
    'min_samples_leaf': [0.008,0.02,0.04], # Número mínimo de amostras necessárias para estar em um nó folha.
}

# Definindo o objeto GridSearchCV
grid = GridSearchCV(model,
                    param_grid,
                    cv=5,
                    scoring='roc_auc',  # 'accuracy' 'precision' 'recall' 'f1'
                    n_jobs=-1)

# Treinando o modelo com o grid search
grid.fit(X_train, y_train)

# Exibindo os melhores parâmetros encontrados pelo grid search
print("Melhores Parâmetros: ", grid.best_params_)

# Exibindo a melhor pontuação (score) atingida pelo modelo com os melhores parâmetros
print("Melhor AUC: ", grid.best_score_)

# Utilizando o melhor modelo para fazer previsões
predictions = grid.best_estimator_.predict(X_test)

best_model_dt = grid.best_estimator_

Melhores Parâmetros:  {'criterion': 'entropy', 'max_depth': 4, 'min_samples_leaf': 0.008, 'min_samples_split': 0.04, 'splitter': 'random'}
Melhor AUC:  0.8401292830246916


In [22]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix, classification_report
modelo_final = DecisionTreeClassifier(criterion='entropy', max_depth=4, min_samples_leaf=0.008, min_samples_split=0.04, splitter='random', random_state=42)
modelo_final.fit(X_train,y_train)

predict = modelo_final.predict(X_test)

print(confusion_matrix(y_test, predict))
print(accuracy_score(y_test, predict))
print(classification_report(y_test, predict))




[[149   8]
 [ 51  60]]
0.7798507462686567
              precision    recall  f1-score   support

           0       0.74      0.95      0.83       157
           1       0.88      0.54      0.67       111

    accuracy                           0.78       268
   macro avg       0.81      0.74      0.75       268
weighted avg       0.80      0.78      0.77       268



In [23]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix, roc_curve, auc

def calculate_metrics(nm_modelo, model, X_train, y_train, X_test, y_test):
    # Fazendo predições
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Calculando as métricas para o conjunto de treino
    accuracy_train = accuracy_score(y_train, y_train_pred)
    precision_train = precision_score(y_train, y_train_pred)
    recall_train = recall_score(y_train, y_train_pred)
    auc_roc_train = roc_auc_score(y_train, model.predict_proba(X_train)[:, 1])

    # Calculando o Índice Gini e Estatística KS para o conjunto de treino
    probabilities_train = model.predict_proba(X_train)[:, 1]
    df_train = pd.DataFrame({'true_labels': y_train, 'predicted_probs': probabilities_train})
    df_train = df_train.sort_values(by='predicted_probs', ascending=False)
    df_train['cumulative_true'] = df_train['true_labels'].cumsum() / df_train['true_labels'].sum()
    df_train['cumulative_false'] = (1 - df_train['true_labels']).cumsum() / (1 - df_train['true_labels']).sum()
    ks_statistic_train = max(abs(df_train['cumulative_true'] - df_train['cumulative_false']))
    gini_index_train = 2 * auc_roc_train - 1

    # Calculando as métricas para o conjunto de teste
    accuracy_test = accuracy_score(y_test, y_test_pred)
    precision_test = precision_score(y_test, y_test_pred)
    recall_test = recall_score(y_test, y_test_pred)
    auc_roc_test = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])

    # Calculando o Índice Gini e Estatística KS para o conjunto de teste
    probabilities_test = model.predict_proba(X_test)[:, 1]
    df_test = pd.DataFrame({'true_labels': y_test, 'predicted_probs': probabilities_test})
    df_test = df_test.sort_values(by='predicted_probs', ascending=False)
    df_test['cumulative_true'] = df_test['true_labels'].cumsum() / df_test['true_labels'].sum()
    df_test['cumulative_false'] = (1 - df_test['true_labels']).cumsum() / (1 - df_test['true_labels']).sum()
    print(df_test)
    ks_statistic_test = max(abs(df_test['cumulative_true'] - df_test['cumulative_false']))
    gini_index_test = 2 * auc_roc_test - 1

    # Criando o DataFrame com as métricas calculadas
    metrics_df = pd.DataFrame({
        'Algoritmo': [nm_modelo, nm_modelo],
        'Conjunto': ['Treino', 'Teste'],
        'Acuracia': [accuracy_train, accuracy_test],
        'Precisao': [precision_train, precision_test],
        'Recall': [recall_train, recall_test],
        'AUC_ROC': [auc_roc_train, auc_roc_test],
        'GINI': [gini_index_train, gini_index_test],
        'KS': [ks_statistic_train, ks_statistic_test]
    })

    return metrics_df


# Uso da função
metrics_df = calculate_metrics('DecisionTreeClassifier',modelo_final, X_train, y_train, X_test, y_test)
metrics_df



     true_labels  predicted_probs  cumulative_true  cumulative_false
796            1         1.000000         0.009009          0.000000
457            1         1.000000         0.018018          0.000000
247            1         1.000000         0.027027          0.000000
412            1         1.000000         0.036036          0.000000
535            1         1.000000         0.045045          0.000000
..           ...              ...              ...               ...
250            0         0.119816         0.990991          0.980892
165            1         0.119816         1.000000          0.980892
388            0         0.119816         1.000000          0.987261
363            0         0.119816         1.000000          0.993631
800            0         0.119816         1.000000          1.000000

[268 rows x 4 columns]


,Algoritmo,Conjunto,Acuracia,Precisao,Recall,AUC_ROC,GINI,KS
0,DecisionTreeClassifier,Treino,0.800963,0.928000,0.502165,0.836354,0.672707,0.553262
1,DecisionTreeClassifier,Teste,0.779851,0.882353,0.540541,0.847249,0.694497,0.601079


In [24]:
X_train['pred'] = modelo_final.predict_proba(X_train)[:, 1]
X_train.head()


,Fare,Age,Sex_male,Pclass,SibSp,Parch,Embarked,pred
445,0.159777,0.044986,1.0,0.0,0.000,0.333333,3,0.290323
650,0.015412,0.367921,1.0,1.0,0.000,0.000000,3,0.119816
172,0.021731,0.007288,0.0,1.0,0.125,0.166667,3,0.443182
450,0.054164,0.447097,1.0,0.5,0.125,0.333333,3,0.207792
314,0.051237,0.535059,1.0,0.5,0.125,0.166667,3,0.207792


In [28]:
abt_train = X_train.join(target, how='inner')
abt_train.tail()


,Fare,Age,Sex_male,Pclass,SibSp,Parch,Embarked,pred,Survived
106,0.014932,0.258608,0.0,1.0,0.000,0.000000,3,0.443182,1
270,0.060508,0.367921,1.0,0.0,0.000,0.000000,3,0.290323,0
860,0.027538,0.509927,1.0,1.0,0.250,0.000000,3,0.207792,0
435,0.234224,0.170646,0.0,0.0,0.125,0.333333,3,0.894737,1
102,0.150855,0.258608,1.0,0.0,0.000,0.166667,3,0.290323,0
